**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../config/config_sing.R")))
show_env()

You are in Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out 
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log 


## Import data

**Check data**

In [2]:
fdiry = file.path(FD_RES, "results", "region", "KS91_K562_ASTARRseq_peak_macs_input")
for (x in dir(fdiry)){cat(x, "\n")}

annotation_ccre 
annotation_chipseq_histone 
annotation_chipseq_tf 
annotation_chipseq_tf_subset 
annotation_chromHMM 
annotation_crispri_e2g 
annotation_crispri_growth 
annotation_crispri_hcrff 
annotation_enhancer_junke 
annotation_enhancer_zscore_junke 
annotation_hic 
annotation_tss_pol2 
annotation_tss_pol2_span 
description.tsv 
enrichment_prepare_annotation 
enrichment_prepare_score_merge 
enrichment_prepare_score_split 
enrichment_results_merge 
enrichment_results_split 
KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed 
KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed.gz 
peak.annotation.screen.tsv 
summary 
tmp 


In [3]:
fdiry = file.path(
    FD_RES, "results", "region", 
    "KS91_K562_ASTARRseq_peak_macs_input", 
    "enrichment_results_merge",
    "annotation_chipseq_histone"
)
fnames = dir(fdiry)
print(fnames)

  [1] "score_merge.astarr.annot.ChIPseq_Histone.H2AFZ_ENCFF213OTI.rds"               
  [2] "score_merge.astarr.annot.ChIPseq_Histone.H3K27ac_ENCFF544LXB.rds"             
  [3] "score_merge.astarr.annot.ChIPseq_Histone.H3K27me3_ENCFF323WOT.rds"            
  [4] "score_merge.astarr.annot.ChIPseq_Histone.H3K27me3_ENCFF801AHF.rds"            
  [5] "score_merge.astarr.annot.ChIPseq_Histone.H3K36me3_ENCFF193ERO.rds"            
  [6] "score_merge.astarr.annot.ChIPseq_Histone.H3K36me3_ENCFF561OUZ.rds"            
  [7] "score_merge.astarr.annot.ChIPseq_Histone.H3K4me1_ENCFF135ZLM.rds"             
  [8] "score_merge.astarr.annot.ChIPseq_Histone.H3K4me1_ENCFF540NGG.rds"             
  [9] "score_merge.astarr.annot.ChIPseq_Histone.H3K4me2_ENCFF749KLQ.rds"             
 [10] "score_merge.astarr.annot.ChIPseq_Histone.H3K4me3_ENCFF122CSI.rds"             
 [11] "score_merge.astarr.annot.ChIPseq_Histone.H3K4me3_ENCFF689QIJ.rds"             
 [12] "score_merge.astarr.annot.ChIPseq_Histone.H3K4me

## Import enrichment results

In [4]:
FOLDERS=c(
    "annotation_genomic",
    "annotation_chipseq_histone",
    "annotation_chipseq_tf"
)

lst = lapply(FOLDERS, function(folder){
    fdiry = file.path(
        FD_RES, "results", "region", 
        "KS91_K562_ASTARRseq_peak_macs_input", 
        "enrichment_results_merge",
        folder
    )
    fname  = "score_merge.overall.*"
    fglob  = file.path(fdiry, fname)
    fpaths = Sys.glob(fglob)
    
    lst = lapply(fpaths, function(fpath){
        dat = readRDS(fpath)
        return(dat)
    })

    dat = bind_rows(lst)
    return(dat)
})

dat = bind_rows(lst)
dat = dat %>% 
    dplyr::select(pathway, pval, padj, log2err, ES, NES, size) %>%
    tidyr::separate(col=pathway, into=c("Annotation", "Label"), sep=":") %>%
    dplyr::mutate(padj  = p.adjust(pval, "BH")) %>%
    dplyr::mutate(nLogP = -log10(padj))

dat_enrich_overall = dat
print(dim(dat))
head(dat)

[1] 781   9


,Annotation,Label,pval,padj,log2err,ES,NES,size,nLogP
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>
1,ccre_v3,"CTCF-only,CTCF-bound",2.897103e-02,3.274439e-02,0.26635066,0.3094418,1.414557,78,1.484863
2,ccre_v3,"dELS,CTCF-bound",1.038735e-02,1.209020e-02,0.38073040,0.4823140,1.766232,19,1.917567
3,ccre_v3,dELS,1.205282e-12,2.736412e-12,0.91011973,0.5170577,2.406543,94,11.562819
4,ccre_v3,"DNase-H3K4me3,CTCF-bound",1.472158e-02,1.698309e-02,0.38073040,0.5601636,1.785218,10,1.769983
5,ccre_v3,DNase-H3K4me3,1.808192e-01,1.916144e-01,0.09721508,0.3959064,1.320317,12,0.717572
6,ccre_v3,DNase-only,1.967574e-02,2.259817e-02,0.35248786,0.3666739,1.572389,48,1.645927


## Save results

In [5]:
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "KS91_K562_ASTARRseq_peak_macs_input", 
    "enrichment_results_merge",
    "summary")
fname = "results.enrich.merge.overall.tsv"
fpath = file.path(fdiry, fname)

dat = dat_enrich_overall
write_tsv(dat, fpath)

In [10]:
fdiry  = file.path(FD_RES, "results", "region", "KS91_K562_ASTARRseq_peak_macs_input", "enrichment_results_merge")
fname  = "score_merge.overall.*"
fglob  = file.path(fdiry, fname)
fpaths = Sys.glob(fglob)
print(length(fpaths))
#basename(fpaths)

[1] 88


In [5]:
fdiry  = file.path(FD_RES, "results", "region", "KS91_K562_ASTARRseq_peak_macs_input", "enrichment_results_merge_chipseq_tf")
fname  = "score_merge_overall_*"
fglob  = file.path(fdiry, fname)
fpaths = Sys.glob(fglob)
print(length(fpaths))
#basename(fpaths)

[1] 733


In [17]:
print(fpaths[1])
print(fpaths[2])

[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/enrichment_results_merge/score_merge_overall_annot_ccre_v3_CTCF_only_CTCF_bound.rds"
[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/enrichment_results_merge/score_merge_overall_annot.ccre_v3.CTCF_only_CTCF_bound.rds"


In [18]:
fpaths[1] == fpaths[2]

[1] FALSE

In [19]:
basename(fpaths[1])

[1] "score_merge_overall_annot_ccre_v3_CTCF_only_CTCF_bound.rds"

In [20]:
basename(fpaths[2])

[1] "score_merge_overall_annot.ccre_v3.CTCF_only_CTCF_bound.rds"

In [14]:
fdiry  = file.path(FD_RES, "results", "region", "KS91_K562_ASTARRseq_peak_macs_input", "enrichment_results_merge")
fname  = "score_merge_overall_*"
fglob  = file.path(fdiry, fname)
fpaths = Sys.glob(fglob)
print(length(fpaths))

lst = lapply(fpaths, function(fpath){
    dat = readRDS(fpath)
    print(fpath)
    print(dim(dat))
    return(dat)
})

dat = bind_rows(lst)

dat_enrich_other = dat
print(dim(dat))
head(dat)

[1] 88
[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/enrichment_results_merge/score_merge_overall_annot_ccre_v3_CTCF_only_CTCF_bound.rds"
[1] 0 8
[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/enrichment_results_merge/score_merge_overall_annot.ccre_v3.CTCF_only_CTCF_bound.rds"
[1] 1 8
[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/enrichment_results_merge/score_merge_overall_annot_ccre_v3_dELS_CTCF_bound.rds"
[1] 0 8
[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/enrichment_results_merge/score_merge_overall_annot.ccre_v3.dELS_CTCF_bound.rds"
[1] 1 8
[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/enrichment_results_merge/score_merge_overall_annot_ccre_v3_dELS.rds"
[1] 0 8
[1] "/data/reddylab

pathway,pval,padj,log2err,ES,NES,size,leadingEdge
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<list>
"ccre_v3:CTCF-only,CTCF-bound",2.897103e-02,2.897103e-02,0.26635066,0.3094418,1.414557,78,"chr16:785365-786016 , chr20:56626306-56626978 , chr16:1150597-1150980 , chrX:48025602-48027305 , chr20:58233920-58234495 , chr11:5600369-5600825 , chr16:487655-488120 , chr11:6269709-6272223 , chrX:48971489-48972805 , chrX:49840468-49841285 , chr11:5507143-5507643 , chr20:57514013-57514715 , chr11:33987858-33988347 , chrX:49291582-49293153 , chr2:61425472-61427136 , chr20:57582887-57583526 , chr11:33808171-33810228 , chr20:57594098-57594776 , chr16:578208-579482 , chr11:5808483-5809339 , chr16:96255-97663 , chr2:60581308-60582350 , chr16:546503-547496 , chrX:48493981-48494574 , chr8:126824093-126824987, chr11:6251053-6252694 , chr16:1057282-1057997 , chr2:60674341-60675636 , chr11:5475885-5476903 , chrX:49626945-49627790 , chr16:813403-814002 , chr11:34492757-34493856 , chr11:33857197-33858390 , chr11:6315738-6316403 , chr20:56577302-56578168 , chr20:58209567-58210686 , chr16:745640-746124 , chr11:61897731-61898477 , chr20:57089380-57090283 , chr20:57596718-57597692 , chr16:733665-734298 , chr16:106793-107475 , chr16:1102925-1103930 , chr11:34791840-34792579 , chr16:363022-365210 , chr11:34512779-34514100 , chr8:127724898-127726838"
"ccre_v3:dELS,CTCF-bound",1.038735e-02,1.038735e-02,0.38073040,0.4823140,1.766232,19,"chrX:49039728-49040229 , chr20:57389114-57391310 , chr16:622004-623228 , chr16:142369-143867 , chrX:49230194-49231405 , chr11:34599585-34601201 , chr11:61869565-61871916 , chrX:49177122-49178250 , chrX:49175582-49176185 , chrX:48631228-48632528 , chr20:57464624-57465376 , chr11:5531048-5531939 , chr16:660456-661697 , chrX:48890603-48892283 , chr11:5195849-5196876 , chr8:127914371-127914665, chr8:127967394-127968596, chrX:48616469-48618242"
ccre_v3:dELS,1.205282e-12,1.205282e-12,0.91011973,0.5170577,2.406543,94,"chr11:5280584-5281311 , chr16:807054-808186 , chr11:34809279-34810196 , chr16:112817-114269 , chr8:127898363-127899746, chr11:33941500-33942221 , chr11:5591550-5591995 , chr11:61834076-61835329 , chr8:128556572-128557393, chr11:5275424-5276195 , chrX:48782524-48783758 , chr8:128176331-128177858, chr11:34603280-34604257 , chr20:57450535-57452546 , chrX:48680411-48680783 , chrX:49147328-49149698 , chr11:61800085-61801113 , chr8:128343440-128344454, chr11:33944510-33945297 , chr16:405242-406077 , chr11:34241932-34243528 , chr11:33936760-33937819 , chr2:61287384-61288774 , chr11:34044347-34045793 , chr11:33912236-33913186 , chrX:48936872-48937767 , chr20:57432673-57433564 , chr11:5287782-5288588 , chr8:127854019-127854900, chrX:48961443-48962257 , chr2:60808237-60808934 , chr16:622004-623228 , chr16:142369-143867 , chr8:128065105-128066356, chr8:128035047-128035766, chr20:56422232-56423039 , chrX:49230194-49231405 , chr20:57414987-57415550 , chr11:5258259-5259344 , chr11:4658480-4659558 , chr16:323762-325231 , chr20:57448688-57449248 , chr2:61465300-61466307 , chrX:49190704-49191897 , chr11:34599585-34601201 , chr2:60806729-60807835 , chr16:252256-253387 , chr11:61869565-61871916 , chr20:58373695-58375015 , chrX:48487040-48488842 , chr16:726514-729450 , chr8:128044551-128045666, chr8:127859401-127860630, chr11:61841459-61842879 , chrX:48793902-48795694 , chr20:57409509-57410825 , chr16:144700-145187 , chr16:578208-579482 , chrX:48631228-48632528"
"ccre_v3:DNase-H3K4me3,CTCF-bound",1.113463e-02,1.113463e-02,0.38073040,0.5601636,1.785218,10,"chr2:60550044-60551083 , chr20:58236706-58237346, chr20:57523756-57526705, chr20:58179062-58180036, chr2:60554041-60555742 , chr16:71295-72412 , chr16:1151778-1153303 , chr11:34357192-34359669, chr11:34173487-34175052, chr16:714464-716381"
ccre_v3:DNase-H3K4me3,1.808192e-01,1.808192e-01,0.09721508,0.3959064,1.320317,12,"chr20:58236706-58237346, chrX:48911082-48913003 , chr16:72695-73044 , chr11:33700194-33701286, chr16:71295-72412 , chr16:629444-631401 , chr16:694114-694522 , chr16:1151778

In [7]:
fdiry  = file.path(FD_RES, "results", "region", "KS91_K562_ASTARRseq_peak_macs_input", "enrichment_results_merge_chipseq_tf")
fname  = "score_merge_overall_*"
fglob  = file.path(fdiry, fname)
fpaths = Sys.glob(fglob)

lst = lapply(fpaths, function(fpath){
    dat = readRDS(fpath)
    return(dat)
})

dat = bind_rows(lst)

dat_enrich_chipseq_tf = dat
print(dim(dat))

[1] 0 8


In [8]:
dat = bind_rows(dat_enrich_other, dat_enrich_chipseq_tf)
dat = dat %>% 
    dplyr::select(pathway, pval, padj, log2err, ES, NES, size) %>%
    tidyr::separate(col=pathway, into=c("Annotation", "Label"), sep=":") %>%
    dplyr::mutate(padj  = p.adjust(pval, "BH")) %>%
    dplyr::mutate(nLogP = -log10(padj))

dat_enrich_total = dat
print(dim(dat))
head(dat)

[1] 763   9


,Annotation,Label,pval,padj,log2err,ES,NES,size,nLogP
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>
1,ccre_v3,"CTCF-only,CTCF-bound",5.194805e-02,5.752738e-02,0.1957890,0.2931971,1.340765,80,1.2401254
2,ccre_v3,"dELS,CTCF-bound",3.978387e-03,4.735584e-03,0.4070179,0.4973922,1.850312,21,2.3246265
3,ccre_v3,dELS,7.026912e-13,1.605250e-12,0.9214260,0.5251367,2.457154,96,11.7944574
4,ccre_v3,"DNase-H3K4me3,CTCF-bound",1.113463e-02,1.285283e-02,0.3807304,0.5601636,1.785218,10,1.8910011
5,ccre_v3,DNase-H3K4me3,1.408591e-01,1.494792e-01,0.1128434,0.3977387,1.365310,14,0.8254193
6,ccre_v3,DNase-only,5.465597e-03,6.445519e-03,0.4070179,0.3858264,1.666815,50,2.1907421


## Save results

In [9]:
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "KS91_K562_ASTARRseq_peak_macs_input", 
    "summary")
fname = "results.enrich.merge.overall.tsv"
fpath = file.path(fdiry, fname)

dat = dat_enrich_total
write_tsv(dat, fpath)